In [1]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
import requests
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib inline

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


### Importing excel files containing Berlin (central) boroughs and their corresponding localities 

In [14]:
df_berlin_boroughs = pd.read_excel("berlin_boroughs.xlsx")
df_berlin_localities = pd.read_excel("berlin_localities.xlsx")

Using the locality names to find their latitude and longitude. 

In [15]:
def get_coordinates_localities(locality, output_as='center'):
    """Function to find coordinates based on locality name"""
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             locality+', Berlin',
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output

Using the get_coordinates_localities function and parsing the coordinates to df_berlin_localities

In [16]:
latitudeCln = []
longitudeCln = []
for index, row in df_berlin_localities.iterrows():
    print(row[0])
    lat, long = get_coordinates_localities(locality=row[0], output_as='center')
    latitudeCln.append(lat)
    longitudeCln.append(long)

df_berlin_localities['Latitude'] = latitudeCln
df_berlin_localities['Longitude'] = longitudeCln


Mitte
Moabit
Hansaviertel
Tiergarten
Wedding
Gesundbrunnen
Friedrichshain
Kreuzberg
Prenzlauer Berg
Weissensee
Blankenburg
Heinersdorf
Karow
Stadtrandsiedlung Malchow
Pankow
Blankenfelde
Buch
Französisch Buchholz
Niederschönhausen
Rosenthal
Wilhelmsruh
Charlottenburg
Wilmersdorf
Schmargendorf
Grunewald
Westend
Charlottenburg-Nord
Halensee
Schöneberg
Friedenau
Tempelhof
Mariendorf
Marienfelde
Lichtenrade
Neukölln
Britz
Buckow
Rudow
Gropiusstadt


Mergin the two dataframes into a single (complete) dataframe containing Borough name, Locality, Latitude and Longitude.

In [17]:
df_berlin_complete = pd.concat([df_berlin_boroughs, df_berlin_localities], axis=1).reindex(df_berlin_boroughs.index)
print(df_berlin_complete.shape)

(39, 4)


In [18]:
df_berlin_complete.head()

,Borough,Locality,Latitude,Longitude
0,Mitte,Mitte,52.517690,13.402376
1,Mitte,Moabit,52.530102,13.342542
2,Mitte,Hansaviertel,52.519123,13.341872
3,Mitte,Tiergarten,52.509778,13.357260
4,Mitte,Wedding,52.550123,13.341970


### Creating the map of Berlin 

Starting with finding the coordinates for Berlin

In [19]:
address = 'Berlin, Germany'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of Berlin are {}, {}.'.format(latitude, longitude))

The decimal coordinates of Berlin are 52.5170365, 13.3888599.


In [20]:
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)

In [21]:
# add markers to map
for lat, lng, borough, locality in zip(df_berlin_complete['Latitude'], df_berlin_complete['Longitude'], df_berlin_complete['Borough'], df_berlin_complete['Locality']):
    label = '{}, {}'.format(locality, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin)

In [22]:
map_berlin

### Using the Foursquare API to find nearby venues 

In [51]:
CLIENT_ID = 'hidden' # your Foursquare ID
CLIENT_SECRET = 'hidden' # your Foursquare Secret
VERSION = '20181020'
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: hidden
CLIENT_SECRET:hidden


In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Locality', 
                  'Locality Latitude', 
                  'Locality Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)
        
    return(nearby_venues)

In [50]:
berlin_venues = getNearbyVenues(names=df_berlin_complete['Locality'], latitudes=df_berlin_complete['Latitude'],
                             longitudes=df_berlin_complete['Longitude'])
berlin_venues.head()

,Locality,Locality Latitude,Locality Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mitte,52.51769,13.402376,Berliner Dom,52.519133,13.401046,Church
1,Mitte,52.51769,13.402376,Schlossplatz,52.517657,13.401196,Plaza
2,Mitte,52.51769,13.402376,Spree Side Gallery am Humboldtforum,52.517543,13.403353,Public Art
3,Mitte,52.51769,13.402376,Hohenzollerngruft,52.518819,13.401067,Cemetery
4,Mitte,52.51769,13.402376,Rathausbrücke,52.516990,13.403934,Bridge


In [26]:
berlin_venues.to_csv("all_venues_berlin.csv")

In [27]:
print(berlin_venues.shape)

(4842, 7)


### Creating function to add certain venues to map

In [28]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Locality'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=3,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

### Positive drivers 

#### Finding nearby offices

In [29]:
berlin_venues_offices = getNearbyVenues(names=df_berlin_complete['Locality'], latitudes=df_berlin_complete['Latitude'],
                                       longitudes=df_berlin_complete['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d124941735')
print(berlin_venues_offices.shape)

(902, 7)


In [ ]:
berlin_venues_offices.to_csv('berlin_venues_offices.csv')

In [30]:
map_berlin_offices = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(berlin_venues_offices, 'green', map_berlin_offices)
map_berlin_offices

#### Finding nearby Universities/Colleges 

In [31]:
berlin_venues_universities = getNearbyVenues(names=df_berlin_complete['Locality'], latitudes=df_berlin_complete['Latitude'],
                                       longitudes=df_berlin_complete['Longitude'], radius=1000, categoryIds='4d4b7105d754a06372d81259')
print(berlin_venues_universities.shape)

(641, 7)


In [19]:
berlin_venues_universities.to_csv('berlin_venues_universities.csv')

In [32]:
map_berlin_universities = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(berlin_venues_universities, 'blue', map_berlin_universities)
map_berlin_universities

#### Finding nearby Shopping Malls 

In [33]:
berlin_venues_shopping_malls = getNearbyVenues(names=df_berlin_complete['Locality'], latitudes=df_berlin_complete['Latitude'],
                                       longitudes=df_berlin_complete['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1fd941735')
print(berlin_venues_shopping_malls.shape)

(74, 7)


In [ ]:
berlin_venues_shopping_malls.to_csv('berlin_venues_shopping_malls.csv')

In [34]:
map_berlin_shopping_malls = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(berlin_venues_shopping_malls, 'orange', map_berlin_shopping_malls)
map_berlin_shopping_malls

### Negative drivers 

#### Finding nearby Food Trucks 

In [35]:
berlin_venues_food_trucks = getNearbyVenues(names=df_berlin_complete['Locality'], latitudes=df_berlin_complete['Latitude'],
                                       longitudes=df_berlin_complete['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1cb941735')
print(berlin_venues_food_trucks.shape)

(81, 7)


In [36]:
berlin_venues_food_trucks.to_csv('berlin_venues_food_trucks.csv')

In [37]:
map_berlin_food_trucks = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(berlin_venues_food_trucks, 'red', map_berlin_food_trucks)
map_berlin_food_trucks

#### Finding nearby burger joints 

In [38]:
berlin_venues_burger_joints = getNearbyVenues(names=df_berlin_complete['Locality'], latitudes=df_berlin_complete['Latitude'],
                                       longitudes=df_berlin_complete['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d16c941735')
print(berlin_venues_burger_joints.shape)

(164, 7)


In [ ]:
berlin_venues_burger_joints.to_csv('berlin_venues_burger_joints.csv')

In [39]:
map_berlin_burger = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(berlin_venues_burger_joints, 'black', map_berlin_burger)
map_berlin_burger

#### Finding nearby Hot Dog joints

In [40]:
berlin_venues_hot_dogs = getNearbyVenues(names=df_berlin_complete['Locality'], latitudes=df_berlin_complete['Latitude'],
                                       longitudes=df_berlin_complete['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d16f941735')
print(berlin_venues_hot_dogs.shape)

(14, 7)


In [ ]:
berlin_venues_hot_dogs.to_csv('berlin_venues_hot_dogs.csv')

In [41]:
map_berlin_hot_dogs = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(berlin_venues_hot_dogs, 'purple', map_berlin_hot_dogs)
map_berlin_hot_dogs

## Creating a new dataframe with

In [42]:
def add_column(start_df, columnTitle, data_df):
    grouped = data_df.groupby('Locality').count()
    
    for n in start_df['Locality']:
        try:
            start_df.loc[start_df['Locality'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            start_df.loc[start_df['Locality'] == n,columnTitle] = 0

In [43]:
df_data = df_berlin_complete.copy()
add_column(df_data, 'Offices', berlin_venues_offices)
add_column(df_data, 'Universities', berlin_venues_universities)
add_column(df_data, 'Shopping Malls', berlin_venues_shopping_malls)
add_column(df_data, 'Food Trucks', berlin_venues_food_trucks)
add_column(df_data, 'Burger Joints', berlin_venues_burger_joints)
add_column(df_data, 'Hot Dogs Joints', berlin_venues_hot_dogs)
df_data.head()

,Borough,Locality,Latitude,Longitude,Offices,Universities,Shopping Malls,Food Trucks,Burger Joints,Hot Dogs Joints
0,Mitte,Mitte,52.517690,13.402376,50.0,50.0,24.0,22.0,12.0,4.0
1,Mitte,Moabit,52.530102,13.342542,49.0,32.0,2.0,1.0,11.0,1.0
2,Mitte,Hansaviertel,52.519123,13.341872,47.0,44.0,1.0,1.0,6.0,1.0
3,Mitte,Tiergarten,52.509778,13.357260,47.0,36.0,1.0,4.0,2.0,1.0
4,Mitte,Wedding,52.550123,13.341970,20.0,49.0,2.0,3.0,4.0,0.0


### Assigning weighting values 

#### Positive drivers (positive weighting)

In [44]:
# I assume that the majority of potential customers will be workers having lunch. Therefore proximity to offices should have a significant positive weight. 
weight_offices = 1.5

# A lot of students eat junk food. Being nearby university related buildings will open a large customer group. Positive weighting, but little less than offices.
weight_universities = 1.0

# Being close to a shopping malls leads to many potential customers. Lower weighting as shopping malls usually has a lot of food courts etc.
weight_shopping_malls = 0.5


#### Negative drivers (negative weighting) 

In [45]:
# Other Food trucks would presumably be direct competetion to the client's food truck. Setting this weight to negative -1.5
weight_food_trucks = -1.5

# The Food truck will mostly serve hamburgers and hot dogs. Meaning, existing burger and hot dog joints will be harsh competitors. Setting negative weight. 
weight_burger_joints = -1.0
weight_hod_dogs = -0.5

## Using the weights to calculate optimal neighborhood

In [46]:
df_weighted = df_data.copy()
df_weighted['Score'] = df_data['Offices']*weight_offices + df_data['Universities']*weight_universities + df_data['Shopping Malls']*weight_shopping_malls + df_data['Food Trucks']*weight_food_trucks + df_data['Burger Joints']*weight_burger_joints + df_data['Hot Dogs Joints']*weight_hod_dogs
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted.head()

,Borough,Locality,Latitude,Longitude,Offices,Universities,Shopping Malls,Food Trucks,Burger Joints,Hot Dogs Joints,Score
21,Charlottenburg-Wilmersdorf,Charlottenburg,52.515747,13.309683,48.0,47.0,5.0,1.0,7.0,0.0,113.0
2,Mitte,Hansaviertel,52.519123,13.341872,47.0,44.0,1.0,1.0,6.0,1.0,107.0
3,Mitte,Tiergarten,52.509778,13.357260,47.0,36.0,1.0,4.0,2.0,1.0,98.5
5,Mitte,Gesundbrunnen,52.550920,13.384846,49.0,27.0,4.0,0.0,4.0,1.0,98.0
22,Charlottenburg-Wilmersdorf,Wilmersdorf,52.487115,13.320330,43.0,36.0,1.0,1.0,5.0,0.0,94.5


In [47]:
df_charlottenburg = df_weighted[df_weighted['Locality']=='Charlottenburg']
df_charlottenburg

,Borough,Locality,Latitude,Longitude,Offices,Universities,Shopping Malls,Food Trucks,Burger Joints,Hot Dogs Joints,Score
21,Charlottenburg-Wilmersdorf,Charlottenburg,52.515747,13.309683,48.0,47.0,5.0,1.0,7.0,0.0,113.0


### Mapping my results 

In [48]:
charlottenburg_latitude = 52.515747
charlottenburg_longitude = 13.309683

In [49]:
map_berlin_results = folium.Map(location=[charlottenburg_latitude, charlottenburg_longitude], zoom_start=14)

berlin_optimal_neighborhood = df_berlin_complete[df_berlin_complete['Locality'] == 'Charlottenburg']

for lat, lng, local in zip(berlin_optimal_neighborhood['Latitude'], berlin_optimal_neighborhood['Longitude'], berlin_optimal_neighborhood['Locality']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='yellow',
        fill_opacity=1).add_to(map_berlin_results)
    
addToMap(berlin_venues_offices[berlin_venues_offices['Locality'] == 'Charlottenburg'], 'green', map_berlin_results)
addToMap(berlin_venues_universities[berlin_venues_universities['Locality'] == 'Charlottenburg'], 'blue', map_berlin_results)
addToMap(berlin_venues_shopping_malls[berlin_venues_shopping_malls['Locality'] == 'Charlottenburg'], 'orange', map_berlin_results)
addToMap(berlin_venues_food_trucks[berlin_venues_food_trucks['Locality'] == 'Charlottenburg'], 'red', map_berlin_results)
addToMap(berlin_venues_burger_joints[berlin_venues_burger_joints['Locality'] == 'Charlottenburg'], 'black', map_berlin_results)
addToMap(berlin_venues_hot_dogs[berlin_venues_hot_dogs['Locality'] == 'Charlottenburg'], 'purple', map_berlin_results)

map_berlin_results